## Working with the US Census Bureau APIs

NaLette M. Brodnax<br>
nalette.brodnax@georgetown.edu 

The [US Census Bureau API](https://www.census.gov/data/developers/data-sets.html) provides a large number of datasets containing social and economic information about different groups of people living in the United States. Because the Census Bureau was one of the first government agencies to make its data accessible via API, using the API often involves navigating a mix of older "legacy" documentation alongside more recently updated webpages. 

### The American Community Survey
The Census Bureau organizes its datasets according to the surveys that it conducts. In order to avoid the time and expense of surveying a sample representative of the entire population every year, the agency varies its survey sampling with respect to time,  geographic scope, and the information collected.  For this tutorial, we will use the American Community Survey (ACS) as an example. 

Every five years, the agency conducts a large survey in order to estimate a set of variables at the neighborhood block level -- the smallest geographic entity available. We will refer to this as the "ACS 5." In the past, every three years, survey data were produced for a smaller sample of geographic entities: those with populations of 20,000 or greater. Although the 3-year ACS surveys are no longer conducted, data from past surveys are still available via the API. And finally, every year, estimates are produced for geographic entities with populations of 65,000 or greater ("ACS 1").

You may find it helpful to review the bureau's guidance on [When to Use 1-year, 3-year, or 5-year Estimates](https://www.census.gov/programs-surveys/acs/guidance/estimates.html).

### Choosing among survey tables
The surveys provide data for each year, but the specific variables may shift depending on the particular survey. In order to protect the privacy of individual respondents, the ACS publishes variables in aggregate form for a given geographic entity (e.g. "Estimate of Total Population", "Estimate of Median Age") as well as cross-tabulations of variables (e.g., "Estimate of Total Population, Female"). These aggregations are provided in what the agency refers to as tables or profiles. 

For example, the [ACS 5 (2009-2018)](https://www.census.gov/data/developers/data-sets/acs-5year.html) includes the following tables and profiles, per the Census Bureau website:

<blockquote>
Detail tables, Subject tables, Data Profiles, and Comparison profiles include the following geographies: nation, all states (including DC and Puerto Rico), all metropolitan areas, all congressional districts (116th congress), all counties, all places, all tracts and block groups.
<ul>
<li>Detailed Tables contain the most detailed cross-tabulations, many of which are published down to block groups. The data are population counts. There are over 20,000 variables in this dataset.</li>
<li>Subject Tables provide an overview of the estimates available in a particular topic.  The data are presented as population counts and percentages.  There are over 18,000 variables in this dataset.</li> 
<li>Data Profiles contain broad social, economic, housing, and demographic information. The data are presented as population counts and percentages. There are over 1,000 variables in this dataset.</li>
<li>Comparison Profiles are similar to Data Profiles but also include comparisons with past-year data.  The current year data are compared with prior 5-Year data and include statistical significance testing.  There are over 1,000 variables in this dataset.</li>
</ul>
</blockquote>

### Finding the right endpoints and variables
Generally, the API has specific **endpoints** for each survey-year-table combination. However, not every survey includes every table. For example, the legacy [ACS 3 (2012-2013)](https://www.census.gov/data/developers/data-sets/acs-3year.2013.html) survey includes only a Detail Table and a Data Profile.

The API documentation is detailed and disaggregated. This means that for a given survey and period combination, you need to find the specific documentation for that survey and time period. Within that page, different years may be provided as tabs.  Unfortunately, the API documentation does not have an easy way to find the data you need. Below are my tips and suggestions.

#### Check available APIs
Check the list of [Available APIs](https://www.census.gov/data/developers/data-sets.html). This will give you a broad overview of the surveys available as well as the years of coverage. The years of coverage will be indicated in parentheses.

#### Search by geographic unit of analysis 
Narrow down the subset of surveys based on your geographic **unit of analysis**. Since block-level data are generated for only the ACS 5, knowing that you intend to collect the most granular data will narrow your search to only those surveys. Common geographies include blocks, census tracts, counties, and states. Note that the agency does not have a comprehensive geographic entity for cities, although it does have "places" as well as metropolitan statisticals areas (MSAs). For example, the Chicago MSA would include the entire population of the Chicago metro area rather than just the population living within the borders of Chicago "proper."

Speaking of **geographic entities**, the bureau uses a set of codes defined by the [Federal Information Processing Standards](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards) (FIPS). Each geographic entity has a unique FIPS code, which can be used to cross-reference data from different surveys. You can get a list of the [FIPS codes by geographic entity type](https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html) from the Census Bureau website. Exercise caution when importing and exporting codes using Pandas, as many have leading zeroes that can be lost in translation. In order to preserve the FIPS codes, I recommend importing data as having the "object" (string) datatype and then converting numeric variables as such at a later point.

#### Check years of coverage in the Discovery Tool
Use the [Census Data API Discovery Tool](https://www.census.gov/data/developers/updates/new-discovery-tool.html). The Discovery Tool is simply a large table with a list of every survey as well as its years of coverage, geographies, variables, documentation link, and endpoint. Once I've identified the appropriate survey, I often go here to see the surveys conducted by year as well as the relevant endpoints. The table is sorted by year, so the most recent surveys will be towards the bottom of the page

#### Search for required and optional variables of interest 
From either the survey documentation page or the Discovery Tool, you can find the **list of variables**. This can be challenging to work with due to both the sheer number of variables and inconsistency over time. The same variable or cross-tabulation may not exist for every year, even across the same survey and table combination. In some cases, the agency may have [guidance](https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data/2018.html) on making comparisons over time. I often start by searching the list of variable labels via Ctrl+F to identify the variables that I need. Once you have identified a variable of interest, you will need to save the variable Name (e.g., "B01001_002E"). You will later pass this name as an argument in your API query. The variable list will also indicate whether a given variable is required as part of the query.

### Constructing an API query
You can use the RAPTOR model as you would with other APIs. Each API query must include an **API key**, which you can register for on the [US Census Developers](https://www.census.gov/developers/) site. You should receive the key immediately. You can also join a mailing list to receive periodic updates.

In [ ]:
import requests
import json
import pandas as pd

#### REVIEW
For this example, I will collect county-level data for the year 2012 using the ACS 5. In reviewing the [documentation](https://www.census.gov/data/developers/data-sets/acs-5year.2012.html) page for this survey, I have clicked on the 2012 tab for further details. The [variables](https://api.census.gov/data/2012/acs/acs5/profile/variables.html) I want to retrieve are included in the Data Profiles.
 * *NAME* - Geography
 * *DP02_0066PE* - Percent!!EDUCATIONAL ATTAINMENT!!Percent high school graduate or higher
 * *DP03_0005PE* - Percent!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Unemployed
 
Notice that the third variable, DP03_0005PE is a cross-tabulation: the intersection of (1) individuals in the labor force, (2) individuals who are not in the military, and (3) individuals who were unemployed in that year.

The documentation also provides me with the relevant API endpoint.

In [ ]:
host = 'http://api.census.gov/data/2012/acs/acs5/profile'

Note: This endpoint looks fairly straightforward, so perhaps we can simply change the year if we want to get the same data for a different year. This may or may not be the case! For example, the same endpoint for 2013 ends with `2013/acs5/profile` -- it's missing `acs/` after the year. Always double check the endpoints so that you are querying the right data.

#### ACCESS

In this step I will first import the API key needed to access the data. I recommend that you never save credentials in a script. I save my API key in a plain text file and then read the key into the environment when needed.

In [ ]:
local_file = 'census_auth.txt'
with open(local_file) as txtfile:
    auth = txtfile.read()

Next, I will build the API query request. There are three components to the query:
 * The `var` variable stores the list of variables.
 * The `geo` variable stores the geographic unit as well as which entities to include. Note: The APIs have query limitations that are not well documented. As a result, if you attempt to query too much data with one call, you will get an error. The error may or may not be informative (e.g., "Bad request").
 * The `auth` variable stores my API key.
 
Depending on the size of your query, it may take awhile to run.

In [ ]:
var = "NAME,DP02_0066PE,DP03_0005PE"
geo = "county:*"
url = host + "?get=" + var + "&for=" + geo + "&key=" + auth

# check the HTTP response code
response = requests.request('GET', url)
print(response)

#### PARSE
In this step I will convert the data from JSON to a python dictionary.

In [ ]:
data = response.json()
data[:5]

#### TRANSFORM
Upon reviewing the data, I can see that the structure is a set of nested lists rather than a dictionary. In order to work with the data in a tidy format, I need to exclude the header row. I will also give the columns new names. Note that the state and county columns indicate their respective FIPS codes. 

In [ ]:
column_names = ['county_name', 'pct_highschool', 'pct_unemployed',
               'state_fips', 'county_fips']

counties = pd.DataFrame(data[1:], columns=column_names)
counties.head()

#### STORE

Finally, I will store the data in a CSV file.

In [ ]:
counties.to_csv('../data/acs5_counties.csv', index=False)

I hope that this tutorial has been helpful. Please email me at nalette.brodnax@georgetown.edu if you have comments or suggestions.